# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"].values

# Display a sample of y
y[:5]

array([0, 0, 0, 1, 0], dtype=int64)

In [6]:
# Define features set X by selecting all columns but credit_ranking
processed_df = loans_df.copy()
X = processed_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
X.head(5)

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)


# Fit the scaler to the features testing dataset
scaler.fit(X_test)

# Transform the features training and testing datasets
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
number_of_features = X_train.shape[1]

# Review the number of features
number_of_features

11

In [13]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer_1 = 11

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer_2 = 8

# Define the number of neurons in the output layer
output_neurons = 1

In [14]:
# Create the Sequential model instance
seq_model = Sequential()

# Add the first hidden layer
seq_model.add(Dense(units=hidden_nodes_layer_1, input_dim=number_of_features, activation='relu'))

# Add the second hidden layer
seq_model.add(Dense(units=hidden_nodes_layer_2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
seq_model.add(Dense(units=output_neurons, activation='sigmoid'))


In [15]:
# Display the Sequential model summary
seq_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                132       
                                                                 
 dense_1 (Dense)             (None, 8)                 96        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 237
Trainable params: 237
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [16]:
# Compile the Sequential model
seq_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Fit the model using 50 epochs and the training data
seq_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 1s 2ms/step - loss: 0.7082 - accuracy: 0.5363
Epoch 2/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6595 - accuracy: 0.6614
Epoch 3/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6268 - accuracy: 0.7181
Epoch 4/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5968 - accuracy: 0.7314
Epoch 5/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5726 - accuracy: 0.7339
Epoch 6/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5548 - accuracy: 0.7306
Epoch 7/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5425 - accuracy: 0.7356
Epoch 8/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5328 - accuracy: 0.7373
Epoch 9/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5261 - accuracy: 0.7398
Epoch 10/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5211 - accuracy: 0.7415
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [18]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = seq_model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 [==============================] - 0s 1ms/step - loss: 0.5246 - accuracy: 0.7425
Loss: 0.5246030688285828, Accuracy: 0.7425000071525574


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [19]:
# Set the model's file path
model_file_output_path = "model/student_loans.keras"

# Export your model to a keras file
seq_model.save(model_file_output_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [21]:
# Set the model's file path
saved_model_path = "model/student_loans.keras"

# Load the model to a new object
loaded_model = tf.keras.models.load_model(saved_model_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [22]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
predictions[:5]

13/13 [==============================] - 0s 1ms/step


array([[0.22793062],
       [0.3580822 ],
       [0.7579843 ],
       [0.72637403],
       [0.9862643 ]], dtype=float32)

In [23]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(predictions, columns=["Prediction"])
predictions_df["Prediction"] = predictions_df["Prediction"].apply(lambda x: 1 if x >= 0.5 else 0)

predictions_df

,Prediction
0,0
1,0
2,1
3,1
4,1
...,...
395,1
396,0
397,1
398,0


### Step 4: Display a classification report with the y test data and predictions

In [25]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["Prediction"]))

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       188
           1       0.78      0.72      0.75       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.75      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

To build a recommendation system for student loans, the following data would be needed:
1. **Degree and Level**: major, course load, Undergrad, Grad, Doctoral.
2. **Financial Data**: Income, credit score, existing debts.
3. **Loan Details**: Interest rates, repayment terms, loan amounts, private, federal.
4. **Demographic Data**: Age, location, employment status and history.
5. **Historical Data**: Past loan performance, default rates.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

Based on the the data, the model would use **content-based filtering**. This is because the recommendations are based on the an individual student's attributes, e.g. financial situation, degree and level, demographic information, and loan details. This data is suitable for leveraging the characteristics of each student to recommend the most appropriate loan options.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

1. **Data Privacy**: Given the sensitivity of the informaation about the student's financial and historical data, privacy and security is a challenge. Unauthorized access to personal and financial data can lead to the potential of identity theft and other privacy violations.

2. **Bias and Fairness**: The recommendation system must be designed to avoid cross-contamination that could result in biasness by  unfairly disadvantaging groups of students in a negative category. Measuring fairness and equity in loan recommendation is important to prevent improper categorization and classification of students.

The challenges above, if not addressed, can have a long-lasting impact on the trustworthiness and integrity of the recommendation system.

